## Step1:
Create an excel file which contains the ward IDs

In [10]:
# Import packages

import os
import glob
import pandas as pd

# Set the directory which contains the shapefiles

os.chdir("/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/SA_LEVEL4/")

# Extract the ward IDs from the ward shapefile names

extension = 'shp'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

IDS = []
for i in all_filenames:
    I = int(i.strip('ADM4_PCODE_ZA.shp'))
    IDS.append(I)
IDS = list(IDS)

pd.DataFrame(IDS).to_excel('/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/Excelfiles/WARD_ID.xlsx')

## Step 2
Open the saved excelfile

In [11]:
import pandas as pd
IDSS = pd.read_excel('/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/Excelfiles/WARD_ID.xlsx')
IDSS = list(IDSS[0])
col_names = [int(i) for i in IDSS]

## Step 3
Connect to Google earth Engine

In [13]:
# import Google earth engine module
import ee

# Trigger the authentication flow.
ee.Authenticate()

#Authenticate the Google earth engine with google account
ee.Initialize()

Enter verification code: 4/1AX4XfWhazEhJqHFVSJuoVMb2nIebuR25l1VFwuMRQXEdPXiZaFL6nHKws_0

Successfully saved authorization token.


## Step 4
Install geemap and use the list of ward IDs to collect the rainfall data from CHIRPS.

In [17]:
!pip install geemap
import geemap

In [18]:
!pwd

/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/SA_LEVEL4


## Step 5
Choose one ward and test the data collection for one ward

In [21]:
col_names[2]

6372040

In [27]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
table = geemap.shp_to_ee("/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/SA_LEVEL4/ADM4_PCODE_ZA6372040.shp")
  
def yearlyRainfall(year):
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate   = startDate.advance(1, 'year')
    filtered  = chirps.filter(ee.Filter.date(startDate, endDate))
    total     = filtered.reduce(ee.Reducer.sum())
    stats     = total.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': table,
      'scale': 5000,
    })

    f = ee.Feature(None, {
      'year': year,
      'precipitation': stats.get('precipitation_sum')
    })
    return f

years = ee.List.sequence(1981, 2021)
rainfallYears = ee.FeatureCollection(
years.map(yearlyRainfall))

# Save the rainfallYears in google drive as csv files

task = ee.batch.Export.table.toDrive(**{
'collection': ee.FeatureCollection(rainfallYears),
'folder': 'Rainfall2',
'description': '6372040',
'fileFormat': 'CSV'}) 
task.start()

# Alternatively export rainfallYears to excel and save in a folder in local machine

prec = []
for i in range(41):
    prec.append(ee.FeatureCollection(rainfallYears).getInfo()['features'][i]['properties']['precipitation'])
pd.DataFrame(prec).to_excel('/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/chirps/6372040.xlsx')

ValueError: DataFrame constructor not properly called!

## Step 6
Loop through all wards to collect the chirps data for the whole country

In [37]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
for i in col_names:
    
    table = geemap.shp_to_ee("/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/SA_LEVEL4/ADM4_PCODE_ZA"+str(i)+".shp")
    
    def yearlyRainfall(year):
        startDate = ee.Date.fromYMD(year, 1, 1)
        endDate   = startDate.advance(1, 'year')
        filtered  = chirps.filter(ee.Filter.date(startDate, endDate))
        total     = filtered.reduce(ee.Reducer.sum())
        stats     = total.reduceRegion(**{
          'reducer': ee.Reducer.mean(),
          'geometry': table,
          'scale': 5000,
        })

        f = ee.Feature(None, {
          'year': year,
          'precipitation': stats.get('precipitation_sum')
        })
        return f

    years = ee.List.sequence(1981, 2021)
    rainfallYears = ee.FeatureCollection(
    years.map(yearlyRainfall))

    # Save the rainfallYears in google drive as csv files
    
    task = ee.batch.Export.table.toDrive(**{
    'collection': ee.FeatureCollection(rainfallYears),
    'folder': 'Rainfall',
    'description': str(i),
    'fileFormat': 'CSV'}) 
    task.start()
    
    # Alternatively export rainfallYears to excel and save in a folder in local machine

    prec = []
    for i in range(41):
        prec.append(ee.FeatureCollection(rainfallYears).getInfo()['features'][i]['properties']['precipitation'])
    pd.DataFrame(prec).to_excel('/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/chirps/'+str(i)+'.xlsx')

## Step 7
Merge the excel files into one single file

In [ ]:
# Locate the directory in which the chirps dataset are
os.chdir("/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/chirps/")

extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

col_names = [i.replace('.xlsx','') for i in all_filenames]

#combine all files in a list
combined_excel = pd.concat([pd.read_excel(f) for f in all_filenames],axis=1)
Updated_excel  = combined_excel.drop(['system:index','.geo'],inplace=True,axis=1)
#export to excel
combined_excel.to_excel( "/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/Excelfiles/Chirps_Rainfall_1981_2021_wards_SA.xlsx", index=False, encoding='utf-8-sig')

In [ ]:
Updated = pd.read_excel('/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/Excelfiles/Chirps_Rainfall_1981_2021_wards_SA.xlsx')
Updated.head()

In [ ]:
#Updated = pd.read_csv('Updated.csv')
Updated.columns = col_names
pd.set_option('max_columns', None)
Updated.head()

In [ ]:
Updated2 = [int(i) for i in Updated.columns] 
Updated3=sorted(Updated2)
Updated3

In [ ]:
Updated4 = [str(i) for i in Updated3]
UpdatedFinal = Updated[Updated4]
UpdatedFinal.tail()

In [ ]:
UpdatedFinal.to_excel("/Users/noe/Documents/Data_science/Geospatial_DS/Rainfall_Data_Collection/Excelfiles/Chirps_Rainfall_1981_2021_wards_SA_Final.xlsx")